# Experiments driver for NATOPS

This notebook collects the experiments performed for TIME 2025.

In [ ]:
import Pkg
using Serialization: serialize, deserialize

# activate the general environment of ModalAssociationRules.jl package
Pkg.activate(joinpath("..", "..", ".."))

# a collection of utilities
include(joinpath("..", "experiments-driver.jl"));

  Activating project at `~/.julia/dev/ModalAssociationRules`


In [26]:
# algorithm to be used for mining
miningalgo = apriori

# distance function to compare a time series and a motif of the same length; 
# we suggest to use Z-Normalized Euclidean distance if you are not interested in the scale
# but just in identifying common shapes between signals.
expdistance = (x, y) -> zeuclidean(x, y) |> first;

In [ ]:
# load NATOPS dataset
X, y = load_NATOPS();

# also, insert a new column representing the difference in height between the thumb and 
# the middle finger tip of the right hand;
# this is useful to describe "All clear" and "Not clear" classes.
insertcols!(X, 25, "ΔY[Thumb r and Hand tip r]" => X[:,5]-X[:,23]);

In [28]:
# separate all the classes in separate collections

# I have command
IHCC = reduce(vcat, [X[1:30, :], X[(180+1):(180+30), :]]);
# All clear
ACC = reduce(vcat, [X[31:60, :], X[(180+31):(180+60), :]]);
# Not clear
NCC = reduce(vcat, [X[61:90, :], X[(180+61):(180+90), :]]);
# Spread wings
SWC = reduce(vcat, [X[91:120, :], X[(180+91):(180+120), :]]);
# Fold wings
FWC = reduce(vcat, [X[121:150, :], X[(180+121):(180+150), :]]);
# Lock wings
LWC = reduce(vcat, [X[151:180, :], X[(180+151):(360), :]]);

In [29]:
# name for each variable;
# this will be useful later when printing during the snippets labeling

variablenames = [
    "X[Hand tip l]", "Y[Hand tip l]", "Z[Hand tip l]",
    "X[Hand tip r]", "Y[Hand tip r]", "Z[Hand tip r]",
    "X[Elbow l]", "Y[Elbow l]", "Z[Elbow l]",
    "X[Elbow r]", "Y[Elbow r]", "Z[Elbow r]",
    "X[Wrist l]", "Y[Wrist l]", "Z[Wrist l]",
    "X[Wrist r]", "Y[Wrist r]", "Z[Wrist r]",
    "X[Thumb l]", "Y[Thumb l]", "Z[Thumb l]",
    "X[Thumb r]", "Y[Thumb r]", "Z[Thumb r]",
    "ΔY[Thumb r and Hand tip r]"
];

## I have command

In [ ]:
# run this cell if you need to create your literals, that is,
# to extract snippets from time series and give them a name.

ids, motifs, featurenames = label_motifs(
    IHCC, 
    vcat(collect(4:6), collect(10:12)), # consider only right hand and right elbow,
    "NATOPS-IHCC"
)

In [ ]:
# run this cell if you already created your literals

ids, motifs, featurenames = load_motifs("NATOPS-IHCC")

In [ ]:
logiset, miner = initialize_experiment(ids, motifs, featurenames, IHCC)
experiment!(miner, "i_have_command")

In [ ]:
variables = [
    VariableDistance(id, m, distance=expdistance, featurename=name)
    for (id, m, name) in zip(ids, motifs, featurenames)
]

propositionalatoms = [
    Atom(ScalarCondition(v, <=, __suggest_threshold(v, IHCC; _percentile=10)))
    for v in variables
]

atoms = reduce(vcat, [
    propositionalatoms,
    diamond(IA_A).(propositionalatoms),
    diamond(IA_B).(propositionalatoms),
    diamond(IA_E).(propositionalatoms),
])

_items = Vector{Item}(atoms)

_itemsetmeasures = [(gsupport, 0.1, 0.1)]
_rulemeasures = [
    (gconfidence, 0.1, 0.1),
    (glift, 0.0, 0.0), # we want to compute lift, regardless of a threshold
]

logiset, miner = initialize_experiment(IHCC);
experiment!(miner, "i_have_command")